In [6]:
import os
import sys

sys.path.insert(0, '../src')

import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from model import InpainTor
from losses import CompositeLoss
from dataset import RORDDataset
import optuna

In [7]:
# Dataset directory configuration
external_disk_path = "/media/tiagociiic/easystore"
split = 'debug'
dataset_dir = os.path.join(external_disk_path, "RORD")
train_dir = os.path.join(dataset_dir, split)
print(f"Dataset directory: {dataset_dir}")

Dataset directory: /media/tiagociiic/easystore/RORD


In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the dataset
train_dataset = RORDDataset(root_dir=dataset_dir, split='debug', image_size=[512, 512])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)


def train_epoch(model, optimizer, criterion, dataloader):
    model.train()
    total_loss = 0
    for batch in dataloader:
        inputs, seg_gt, inpaint_gt = [batch[k].to(device) for k in ['image', 'mask', 'gt']]
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, seg_gt, inpaint_gt)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)


def objective(trial):
    model = InpainTor().to(device)
    criterion = CompositeLoss(seg_loss=torch.nn.NLLLoss(), inpaint_loss=torch.nn.MSELoss(), lambda_=0.5)

    # Suggested learning rate
    lr = trial.suggest_float("lr", 1e-6, 1e-1)

    optimizer = optim.AdamW(model.parameters(), lr=lr)

    # Train the model
    n_epochs = 3
    losses = []
    for epoch in range(n_epochs):
        loss = train_epoch(model, optimizer, criterion, train_loader)
        losses.append(loss)

    return sum(losses) / len(losses)  # Loss averaged over all epochs


# Optuna study
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

# Best learning rate and loss
best_lr = study.best_params['lr']
best_loss = study.best_value

print(f"Best learning rate: {best_lr}")
print(f"Best loss: {best_loss}")

final_model = InpainTor().to(device)
final_optimizer = optim.AdamW(final_model.parameters(), lr=best_lr)

2024-07-02 08:32:19.417 | INFO     | dataset:__init__:94 - Loading image, ground truth, and mask files...
2024-07-02 08:32:19.419 | INFO     | dataset:__init__:99 - Loading image files from: /media/tiagociiic/easystore/RORD/debug/img, ground truth files from: /media/tiagociiic/easystore/RORD/debug/gt, and mask files from: /media/tiagociiic/easystore/RORD/debug/multiclass_mask
2024-07-02 08:32:19.422 | INFO     | dataset:__init__:110 - Found 10 image files
2024-07-02 08:32:19.422 | INFO     | dataset:__init__:111 - Found 10 ground truth files
2024-07-02 08:32:19.422 | INFO     | dataset:__init__:112 - Found 10 mask files
2024-07-02 08:32:19.423 | INFO     | dataset:__init__:133 - Found 10 image files, 10 ground truth files, and 10 mask files
2024-07-02 08:32:19.423 | INFO     | dataset:__init__:140 - Found 10 common files in the directories
2024-07-02 08:32:19.425 | INFO     | dataset:__init__:162 - Found 10 valid samples out of 10 total samples
[I 2024-07-02 08:32:19,425] A new study c

Best learning rate: 0.09806447731899928
Best loss: 1.8408315844006007
